<a href="https://colab.research.google.com/github/michalis0/DataMining_and_MachineLearning/blob/master/week5/Ridge_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear regression: Standardiztion, Ridge regression, Cross-validation, and more.

In this notebook, we are going to tackle the problem of predicting house prices in more details. We will follow the following steps:

    1- Preprocessing the data
        1.1- One-hot encoding of the categorical columns
        1.2- Feature engineering: selecting a subset of columns based on their correlation with the target variable.
    2- Splitting the data into training and test sets.
    3- Normalization of the data
    4- Training a ridge regression model
    5- Cross-validation to find the best regularizer hyper-parameter
    6- Evaluating the model on the test data

In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
%load_ext autoreload
%autoreload 2
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
sns.set_style("white")

## 0- Loading the data

In [ ]:
housing_data = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataMining_and_MachineLearning/master/week5/data/processed_housing_data.csv")
housing_data.head()

In [ ]:
housing_data.shape

In [ ]:
print(housing_data.dtypes)

## 1- Preprocessing

### 1.1- One-hot encoding of the categorical columns

We can observe that we have non-numerical columns in our dataset. We have to get rid of them and convert them to numerical values before feeding the data to the model. These columns actuaclly have categorical values, meaning that they only take values from a fixed set of values. For example, let's take a look at the `MSZoning` column.

In [ ]:
housing_data["MSZoning"].value_counts()

We can convert such columns to numerical values using one-hot encoding.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
mszoning_ohe = ohe.fit_transform(housing_data[["MSZoning"]])

In [ ]:
mszoning_ohe

In [ ]:
# order of the categories
ohe.categories_

In [ ]:
# looking at the first rwo of the matrix
mszoning_ohe[0].todense()

As you can see the one-hot encoder returns an array of `number of datapoints` $\times$ `number of categoreis`. Each column of this array corresponds to a single category and if it is one, it indicates the categorical value for that data point.

However, with this representation we have to convert the resulting array to a dataframe and also name each column of the resulting dataframe according to the categorical value. With the `get_dummies` method from pandas we can actually do the one-hot encoding more easily.


In [ ]:
mszoning_ohe = pd.get_dummies(housing_data[["MSZoning"]])
mszoning_ohe.head()

so let's extract all the categorical columns and encode them all at once.

In [ ]:
df_types = pd.DataFrame(housing_data.dtypes).reset_index()
string_columns = df_types[df_types[0]=='object']['index']
non_string_columns = df_types[df_types[0]!='object']['index']

In [ ]:
print(string_columns)

In [ ]:
dummy_df = pd.get_dummies(housing_data[string_columns])
# creating the ultimate dataframe where all categorical columns are one-hot encoded
df = pd.concat([dummy_df,housing_data[non_string_columns]], axis=1)

In [ ]:
df.shape

In [ ]:
df.dtypes.value_counts()

### 1.2- Feature engineering

Right now there are almost 231 features. In the following we will try to decrease the number of features based on their correlation to the target.


In [ ]:
df_corr = abs(df.corr()).sort_values(by='SalePrice', ascending=False)[['SalePrice']]
df_corr[df_corr['SalePrice']>0.4]

In [ ]:
# let's only keep the columns with a correlation greater than 0.4
df_small = df[df_corr[df_corr['SalePrice']>0.4].index.tolist()]
df_small.shape

Now we decreased the number of features to 24. 

### 2- Splitting the data into training and test sets.

Next we split the data to training and test sets. The reason we do this is that we want to start the training phase after this step. Any test data that I will introduce during modelling or premodelling, will create a bias in my evaluation metrics. 

In [ ]:
df_small.reset_index(inplace=True,drop=True)

In [ ]:
X = df_small.drop(columns=['SalePrice'])
y = df_small['SalePrice']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [ ]:
X_train.head()

### 3- Normalization of the data

Now we can start with normalize the data. This is helpful to bring all the data to the same scale. We will use the sklearn standard scaler which removes the mean from each featrues and divids it by it standard deviation.

In [ ]:
from sklearn.preprocessing import StandardScaler
# Instantiate Standard Scaler.
ss = StandardScaler()
# fit 
Z_train = ss.fit_transform(X_train)
# transform the df
Z_train = pd.DataFrame(ss.transform(X_train), columns=X_train.columns)

In [ ]:
Z_train.head()


### 4- Training a ridge regression model

Since our data is ready, we can start with the modelling phase.

In [ ]:
from sklearn.linear_model import Ridge

First, Let's visualize the regularization hyper-parameter of ridge and its impact on the regression coefficients.

In [ ]:
def ridge_coefs(X, y, alphas):
    
    # list of coefficients:
    coefs = []
    
    # initiate the model
    ridge_reg = Ridge()
    
    # iterate through the alphas fed into the function:
    for a in alphas:
        
        # reinitiate with the new alpha:
        ridge_reg.set_params(alpha=a)
        
        # refit the model on the provided X, y
        ridge_reg.fit(X, y)
        
        # print the coefficient list
        coefs.append(ridge_reg.coef_)
        
    return coefs
  # this snippet is taken from an online source

In [ ]:
# np.logspace gives us points between specified orders of magnitude on a logarithmic scale. It is base 10.
r_alphas = np.logspace(0, 5, 200)

# Get the coefficients for each alpha for the Ridge, using the function above
r_coefs = ridge_coefs(Z_train, y_train, r_alphas)

In [ ]:
from cycler import cycler

def coef_plotter(alphas, coefs, feature_names, to_alpha, regtype='ridge'):
    
    # Get the full range of alphas before subsetting to keep the plots from 
    # resetting axes each time. (We use these values to set static axes later).
    amin = np.min(alphas)
    amax = np.max(alphas)
    
    # Subset the alphas and coefficients to just the ones below the set limit
    # from the interactive widget:
    alphas = [a for a in alphas if a <= to_alpha]
    coefs = coefs[0:len(alphas)]
    
    # Get some colors from seaborn:
    colors = sns.color_palette("husl", len(coefs[0]))
    
    # Get the figure and reset the size to be wider:
    fig = plt.figure()
    fig.set_size_inches(18,5)

    # We have two axes this time on our figure. 
    # The fig.add_subplot adds axes to our figure. The number inside stands for:
    #[figure_rows|figure_cols|position_of_current_axes]
    ax1 = fig.add_subplot(121)
    
    # Give it the color cycler:
    ax1.set_prop_cycle(cycler('color', colors))
    
    # Print a vertical line showing our current alpha threshold:
    ax1.axvline(to_alpha, lw=2, ls='dashed', c='k', alpha=0.4)
    
    # Plot the lines of the alphas on x-axis and coefficients on y-axis
    ax1.plot(alphas, coefs, lw=2)
    
    # set labels for axes:
    ax1.set_xlabel('alpha', fontsize=20)
    ax1.set_ylabel('coefficients', fontsize=20)
    
    # If this is for the ridge, set this to a log scale on the x-axis:
    if regtype == 'ridge':
        ax1.set_xscale('log')
    
    # Enforce the axis limits:
    ax1.set_xlim([amin, amax])
    
    # Put a title on the axis
    ax1.set_title(regtype+' coefficients\n', fontsize=20)
    
    # Get the ymin and ymax for this axis to enforce it to be the same on the 
    # second chart:
    ymin, ymax = ax1.get_ylim()

    # Add our second axes for the barplot in position 2:
    ax2 = fig.add_subplot(122)
    
    # Position the bars according to their index from the feature names variable:
    ax2.bar(list(range(1, len(feature_names)+1)), coefs[-1], align='center', color=colors)
    ax2.set_xticks(list(range(1, len(feature_names)+1)))
    
    # Reset the ticks from numbers to acutally be the names:
    ax2.set_xticklabels(feature_names, rotation=65, fontsize=12)
    
    # enforce limits and add titles, labels
    ax2.set_ylim([ymin, ymax])
    ax2.set_title(regtype+' predictor coefficients\n', fontsize=20)
    ax2.set_xlabel('coefficients', fontsize=20)
    ax2.set_ylabel('alpha', fontsize=20)
    
    plt.show()
  # this snippet is taken from an online source

In [ ]:
from ipywidgets import *
from IPython.display import display

def ridge_plot_runner(log_of_alpha=0):
    coef_plotter(r_alphas, r_coefs, X.columns, 10**log_of_alpha, regtype='ridge')

interact(ridge_plot_runner, log_of_alpha=(0.1,5,0.2))
print("this snippet is taken from an online source")

### 5- Cross-validation to find the best regularizer hyper-parameter

Thanks to the visualization, we can understand the relationship between the alpha (strength of the regularization coefficient) and coefficients. But how to pick the best alpha? The answer can be found by trial and error. In this case, we will train different models on the training set and pick the model that performs the best. We will do this using cross-validation.

In [ ]:
from sklearn.metrics import r2_score
from sklearn.linear_model import RidgeCV

In [ ]:
# Set up a list of ridge alphas to check.
r_alphas = np.logspace(0, 5, 100)
# Generates 200 values equally between 0 and 5,
# then converts them to alphas between 10^0 and 10^5.

# Cross-validate over our list of ridge alphas.
ridge_model = RidgeCV(alphas=r_alphas, scoring='r2')

# Fit model using best ridge alpha!
ridge_model = ridge_model.fit(Z_train, y_train)

In [ ]:
# Here is the optimal value of alpha
ridge_optimal_alpha = ridge_model.alpha_
ridge_optimal_alpha

### 6- Evaluating the model on the test data

To evaluate the model on the test data, we first need to normalize the test data __using the same normalization that we used to normalize the training data.__

In [ ]:
# transform the test data
Z_test = pd.DataFrame(ss.transform(X_test), columns=X_test.columns)

In [ ]:
# Instantiate the best model.
ridge_opt = Ridge(alpha=ridge_optimal_alpha)

# Fit model.
ridge_opt.fit(Z_train, y_train)

# Generate predictions
ridge_opt_preds = ridge_opt.predict(Z_test)
ridge_opt_preds_train = ridge_opt.predict(Z_train)

# Evaluate model.
print("score on the test data: ", r2_score(y_test, ridge_opt_preds))
print("score on the training data:", r2_score(y_train, ridge_opt_preds_train))

So the conclusion, the R squared value for the test data was 0.82. This is higher than the score from the training dataset which proves that in a dataset that might highly overfit, we achieved to fit our model to the signal rather than the noise. 